In [2]:
%%writefile zigbang.py
import pandas as pd
import requests
import geohash2

def crawling_apt(addr):
    # 동이름 -> 위도,경도
    url = "https://apis.zigbang.com/search?q={}".format(addr)
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    # 위도,경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids
    url = "https://apis.zigbang.com/property/apartments/items?\
domain=zigbang&geohash={}\
&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1".format(geohash)
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["itemId"] for data in datas]
    
    # ids -> items : 200개씩
    dfs = [] 
    for idx in range(0, len(ids), 200):
        start, end = idx, idx + 200
        ids_str = str(ids[start:end]).replace(" ", "")
        url = "https://apis.zigbang.com/property/apartments/items?\
        vritemIds={ids}&itemIds={ids}&citemIds={ids}".format(ids=ids_str)
        response = requests.get(url)
        datas = response.json()["items"]
        item_df = pd.DataFrame(datas)
        columns = [ "itemId", "buildingFloor", "groupedItemFloor", "grossArea", "lat", "lng", "sales", "itemTitle" ]
        item_df = item_df[columns]
        item_df["m2"] = item_df["grossArea"].apply(lambda data: data["m2"])
        item_df["p"] = item_df["grossArea"].apply(lambda data: data["p"])
        item_df.drop(columns=["grossArea"], inplace=True)
        dfs.append(item_df)
    result_df = pd.concat(dfs)
    result_df.reset_index(drop=True, inplace=True)
    return result_df

Overwriting zigbang.py


In [4]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=4030c2d6681500262d88369286ae100406d9871cdc375aefaa8c36402db3802f
  Stored in directory: c:\users\luvu1\appdata\local\pip\cache\wheels\a0\b8\b7\8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent


In [5]:
from fake_useragent import UserAgent

In [6]:
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2656.18 Safari/537.36'

In [8]:
url = "https://finance.daum.net/api/exchanges/summaries"
headers = {
    "user-agent" : UserAgent().chrome,
    "referer": "https://finance.daum.net/exchanges",
}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [14]:
data =  response.json()["data"]
result_df = pd.DataFrame(data)
result_df.head()

,symbolCode,date,currencyCode,currencyName,currencyUnit,country,region,name,recurrenceCount,basePrice,...,changeRate,cashBuyingPrice,cashSellingPrice,ttBuyingPrice,ttSellingPrice,tcBuyingPrice,fcSellingPrice,exchangeCommission,usDollarRate,chartImageUrl
0,FRX.KRWUSD,2020-10-22 12:04:03,USD,달러,1,미국,"{'korName': '아메리카', 'engName': 'America'}",미국 (KRW/USD),162,1135.10,...,0.001412,1154.96,1115.24,1124.00,1146.20,None,None,1.9979,1.0000,{'day': 'https://t1.daumcdn.net/finance/chart/...
1,FRX.KRWJPY,2020-10-22 12:04:02,JPY,엔,100,일본,"{'korName': '아시아', 'engName': 'Asia'}",일본 (KRW/JPY100),162,1083.89,...,0.002757,1102.85,1064.93,1073.27,1094.51,None,None,1.8938,0.9549,{'day': 'https://t1.daumcdn.net/finance/chart/...
2,FRX.KRWCNY,2020-10-22 12:04:02,CNY,위안,1,중국,"{'korName': '아시아', 'engName': 'Asia'}",중국 (KRW/CNY),162,170.52,...,0.001347,179.04,162.00,168.82,172.22,None,None,5.0010,0.1502,{'day': 'https://t1.daumcdn.net/finance/chart/...
3,FRX.KRWEUR,2020-10-22 12:04:02,EUR,유로,1,유로,"{'korName': '유럽', 'engName': 'Europe'}",유로 (KRW/EUR),162,1344.30,...,0.000275,1371.05,1317.55,1330.86,1357.74,None,None,1.4106,1.1843,{'day': 'https://t1.daumcdn.net/finance/chart/...
4,FRX.KRWGBP,2020-10-22 12:04:02,GBP,파운드,1,영국,"{'korName': '유럽', 'engName': 'Europe'}",영국 (KRW/GBP),162,1490.10,...,0.006702,1519.45,1460.75,1475.20,1505.00,None,None,2.0206,1.3127,{'day': 'https://t1.daumcdn.net/finance/chart/...


In [ ]:
# session - 연결 정보 (ram에 저장됨)
# cookie - file (하드디스크에 저장됨)